<a href="https://colab.research.google.com/github/tiago-ds/kpopifier/blob/main/kpopifier_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FONTES: 

Requests: https://docs.python-requests.org/en/latest/

Descobrir o tempo: https://api.sonicAPI.com/analyze/tempo

Mudar o tempo das músicas: http://www.sonicapi.com/docs/api/process-elastique

Dividir a música em partes: https://github.com/deezer/spleeter

Juntar as músicas: https://github.com/jiaaro/pydub



# Instalando as dependências

In [ ]:
!apt install ffmpeg
!pip install spleeter
!pip install pydub
!pip install librosa
!pip install --upgrade kaggle
!pip install gdown

# Vai aparecer um erro, mas é só o python. Funciona, não se preocupa

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
     |████████████████████████████████| 51 kB 5.4 MB/s 
     |████████████████████████████████| 25.3 MB 1.6 MB/s 
     |████████████████████████████████| 14.8 MB 51.1 MB/s 
     |████████████████████████████████| 183 kB 53.0 MB/s 
     |████████████████████████████████| 77 kB 5.2 MB/s 
     |████████████████████████████████| 454.3 MB 14 kB/s 
     |████████████████████████████████| 132 kB 23.8 MB/s 
     |████████████████████████████████| 1.2 MB 26.4 MB/s 
     |████████████████████████████████| 462 kB 4.0 MB/s 
     |████████████████████████████████| 4.0 MB 28.8 MB/s 
     |████████████████████████████████| 58 kB 4.5 MB/s 
     |████████████████████████████████| 57 kB 4.0 MB/s 
     |████████████████████████████████| 79 kB 5.4 MB/s 
     |███████████████████████████████

# Imports

In [ ]:
import requests
import json
import os
import librosa
import time
import IPython
from pydub import AudioSegment
from google.colab import files

# Enviar a música que vai ser kpopificada
Por favor, selecionar um .mp3 ou .wav

In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():
  print(f'\nChosen {fn} to kpopify.')
  sample_file_name = fn[:-4]
  os.environ['SAMPLE_CORRECTED_FILE_NAME'] = f"{sample_file_name}_corrected.wav"
  break

Saving alors_on_danse.mp3 to alors_on_danse.mp3

Chosen alors_on_danse.mp3 to kpopify.


# Descobrindo o tempo da música escolhida, e escolhendo a música base de acordo com isso

In [ ]:
# Descobrindo o tempo da música do usuário
# O warning é normal, pra sinalizar que tá lendo um .mp3 e não um .wav
y, sr = librosa.load(f'{sample_file_name}.mp3')
sample_tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)

# Pequena correção para não ficar um float
sample_tempo = round(sample_tempo)
print(sample_tempo)

# Aqui, é feita a escolha da música base
# Ela é escolhida pra ser o mais próximo da escolhida pelo usuário
musics_tempo = {
    'kingdon_come': (86, '1G0RFXa-M0lwD_K7UYOBlG-a3prhoBXUz'),
    'alcohol_free': (96, '1iLQjt1kwflgHTLheFzgJRRU_1QAtnXtq'),
    'solo': (98, '1TvGqx6FwiK9p7rL6HaZzKEpAciC_9y2u'),
    'dynamite': (112,'1q6wiRbNPRE3nMdtIEUv_fTZ7muXhZpo4'),
    'the_boys': (117, '1YIMwGMaZV9BDNawgcigIcKXwOuDLiubf'),
    'dont_fight_the_feeling': (129, '1yzKPfsrXK_wx0-OrriCjCIzi3Bh6LUOn'),
    'ddu_du_ddu_du': (143, '1DQLWnskeWohL8gQreVRFoq9mdR_w3dnk'),
    'i_cant_stop_me': (151, '1Zz4KUwtAeCixlC0ezunBk6j_MB2C559P'),
    'milkshake': (168, '1TaUBwr1z4hY7qKqmpm9zD1wXTEAT8cu9'),
    'what_is_love': (172, '1EzYHEzszB56A1RUsg4QY5fEXSYV7Qgt5'),
}

# Essa linha de código escolhe a música do dicionário musics_tempo com o tempo mais
# próximo da música escolhida
base_file_name, file_stats = min(musics_tempo.items(), key=lambda x: abs(sample_tempo - x[1][0]))
base_tempo, base_file_id = file_stats[0], file_stats[1]

print("")
print(f"{sample_file_name}: {sample_tempo} BPM ")
print(f"Chosen kpop: {base_file_name} ( {base_tempo} BPM )")
print(f"\n")

# Agora, baixar a música escolhida pra o colab

# Exporta o id do arquivo e o nome dele para o ambiente de execução
os.environ['BASE_FILE_ID'] = base_file_id
os.environ['CORRECTED_FILE_NAME'] = f"{base_file_name}_corrected.wav"
# E baixa o arquivo
!gdown --id $base_file_id

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


117

alors_on_danse: 117 BPM 
Chosen kpop: the_boys ( 117 BPM )


Downloading...
From: https://drive.google.com/uc?id=1F1LB8Y-VD9ByN2_OLJpHU2JxeVysEppM
To: /content/the_boys.mp3
100% 3.63M/3.63M [00:00<00:00, 65.8MB/s]



# Upando os arquivos:


In [ ]:
ACCESSID = "3b2a3c23-7166-438a-b94d-a009e86c577b"

upload_url = f"https://api.sonicapi.com/file/upload?access_id={ACCESSID}"

# Arquivos do POST
sample_file = {'file': open(f"{sample_file_name}.mp3",'rb')}
base_file = {'file': open(f"{base_file_name}.mp3",'rb')}

# Request body
data = {'format': 'json'} 

base_response = requests.request("POST", upload_url, data=data, files=base_file)  
base_file_response_json = json.loads(base_response.text) # Com isso vamos pegar o id do arquivo na sonicAPI
base_file_id = base_file_response_json['file']['file_id']

sample_response = requests.request("POST", upload_url, data=data, files=sample_file)
sample_file_response_json = json.loads(sample_response.text)
sample_file_id = sample_file_response_json['file']['file_id'] 

print(f"Uploaded files' ids: \n\nBase: {base_file_id} \nSample: {sample_file_id}")

Uploaded files' ids: 

Base: 8a9d1bb6-1913-4ee0-afde-aa35f81d5a83 
Sample: a1a6bda5-aa27-41b1-944d-4d5cb48cccd1


# Tirando a média dos tempos, e pegando os arquivos "corrigidos"

In [ ]:
# Fazendo o post pra pegar o arquivo no tempo correto
elastique_api_url = "https://api.sonicAPI.com/process/elastique"

# A média do tempo das duas músicas
mean_tempo = round((base_tempo + sample_tempo) / 2)

print(f'Mean tempo: {mean_tempo}')

# O cálculo do fator de multiplicação dos dois tempos
# Tem um ajuste de duas casas decimais
base_factor = round((mean_tempo / sample_tempo), 2)
sample_factor = round((mean_tempo / base_tempo), 2)

# Condição pra caso um dos tempos for MUITO maior que o outro
# Basicamente, se um tempo / 2 ainda for maior que o outro, a gente divide
if abs(base_factor - sample_factor) >= 1:
  if max(base_factor, sample_factor) == sample_factor:
    sample_factor /= 2
  else:
    base_factor /= 2

# Essa é a correção pra a música 1.
base_data = {
    "access_id": ACCESSID,
    "input_file": base_file_id,
    "tempo_factor": base_factor,
    "format": "json",
    "blocking": "false"
}

# Aí, pega o file_id  da response 
base_file_response = requests.post(elastique_api_url, data = base_data)

if base_file_response.ok:
  base_response_json = json.loads(base_file_response.text)
  corrected_tempo_base_file_id = base_response_json['file']['file_id']
  print(f"Base file corrected. SonicAPI id: {corrected_tempo_base_file_id}")
else:
  print(f"An error ocurred when uploading the base file {base_file_name}.")

# Música 2
sample_data = {
    "access_id": ACCESSID,
    "input_file": sample_file_id,
    "tempo_factor": sample_factor,
    "format": "json",
    "blocking": "false"
}

sample_file_response = requests.post(elastique_api_url, data = sample_data)

if sample_file_response.ok:
  sample_response_json = json.loads(sample_file_response.text)
  corrected_tempo_sample_file_id = sample_response_json['file']['file_id']
  print(f"Sample file corrected. SonicAPI id: {corrected_tempo_sample_file_id}")
else:
  print(f"An error ocurred when uploading the base file {sample_file_name}.")

Mean tempo: 117
Base file corrected. SonicAPI id: 77bdbeab-4181-4a8e-90ce-0cea72d3fb04
Sample file corrected. SonicAPI id: c5d35271-8e2b-4455-93d6-fcb127c6c5db


# Salvando as músicas com o tempo "corrigido"

In [ ]:
# Tem algum problema com a SonicAPI, que se a gente tentar baixar o arquivo 
# logo depois de fazer o POST anterior, ela não funciona
# O console imprime o tamanho do arquivo corrigido, se for 187 bytes, peço que 
# rode esse bloco de novo.

# Salvando a música 1 com o tempo corrigido
download_url = "https://api.sonicAPI.com/file/download"

base_download_request = {
    "access_id": ACCESSID,
    "file_id": corrected_tempo_base_file_id,
    "format": "wav"
}

final_base_file_response = requests.post(download_url, data=base_download_request)
# Salva os arquivos com o tempo corrigido
f = open(f"{base_file_name}_corrected.wav", "wb")
f.write(final_base_file_response.content)
f.close()

print(f"Corrected base tempo file created with size of {os.path.getsize(f'{base_file_name}_corrected.wav')} bytes as {base_file_name}_corrected.wav")

Corrected base tempo file created with size of 40034200 bytes as the_boys_corrected.wav


In [ ]:
# E agora a música 2
sample_download_request = {
    "access_id": ACCESSID,
    "file_id": corrected_tempo_sample_file_id,
    "format": "wav"
}

final_sample_file_response = requests.post(download_url, data=sample_download_request)

f = open(f"{sample_file_name}_corrected.wav", "wb")
f.write(final_sample_file_response.content)
f.close()

print(f"Corrected sample tempo file created with size of {os.path.getsize(f'{sample_file_name}_corrected.wav')} bytes as {sample_file_name}_corrected.wav")

Corrected sample tempo file created with size of 36725804 bytes as alors_on_danse_corrected.wav


# Agora, usa o Spleeter pra dividir as faixas em stems.

In [ ]:
!spleeter separate -p spleeter:4stems -o output "$SAMPLE_CORRECTED_FILE_NAME"
!spleeter separate -p spleeter:4stems -o output "$CORRECTED_FILE_NAME"

INFO:spleeter:File output/alors_on_danse_corrected/vocals.wav written succesfully
INFO:spleeter:File output/alors_on_danse_corrected/drums.wav written succesfully
INFO:spleeter:File output/alors_on_danse_corrected/bass.wav written succesfully
INFO:spleeter:File output/alors_on_danse_corrected/other.wav written succesfully
INFO:spleeter:File output/the_boys_corrected/vocals.wav written succesfully
INFO:spleeter:File output/the_boys_corrected/drums.wav written succesfully
INFO:spleeter:File output/the_boys_corrected/bass.wav written succesfully
INFO:spleeter:File output/the_boys_corrected/other.wav written succesfully


# Depois de fazer o processo com as duas músicas, a gente pode usar isso pra juntar elas:


In [ ]:
sample_vocals = AudioSegment.from_file(f"./output/{sample_file_name}_corrected/vocals.wav", format="wav")
base_bass = AudioSegment.from_file(f"./output/{base_file_name}_corrected/bass.wav", format="wav")
base_drums = AudioSegment.from_file(f"./output/{base_file_name}_corrected/drums.wav", format="wav")
base_other = AudioSegment.from_file(f"./output/{base_file_name}_corrected/other.wav", format="wav")

# Aumentar um pouco a voz da música
louder_vocals = sample_vocals + 5 # 5dB mais alto

# Fazer o overlay das músicas
result_1 = base_bass.overlay(base_drums, position=0)
result_2 = result_1.overlay(base_other, position=0)
result = result_2.overlay(sample_vocals, position=0)

# Export do resultado final
result.export(f"{sample_file_name}_{base_file_name}.mp3", format="mp3")

# Mostrar o player do python
IPython.display.Audio(f"{sample_file_name}_{base_file_name}.mp3")

<_io.BufferedRandom name='alors_on_danse_the_boys.mp3'>

# Se quiser baixar o arquivo:

In [ ]:
files.download(f"{sample_file_name}_{base_file_name}.mp3")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>